### Graph Representation ###
Sparse graph -- Adjacency list

Dense graph -- Adjacency matrix


In [2]:
from typing import List
import collections
import sys
# build an adjacency type graph from adjacency matrix
def makeGraph(edges: List[List[int]]):
    graph = collections.defaultdict(list)
    for u, v in edges:
        graph[u] += [v]
        graph[v] += [u]
    return graph
edges = [[0, 1], [0, 2], [1, 3], [1, 4], [2, 4], [2, 5], [3, 6], [4, 6], [4, 7], [5, 7], [6, 8], [7, 8]]
graph = makeGraph(edges)
'''
the graph looks like below:
  0 -- 2 -- 5
  |    |    |
  1 -- 4 -- 7
  |    |    |
  3 -- 6 -- 8
'''
print(graph)

defaultdict(<class 'list'>, {0: [1, 2], 1: [0, 3, 4], 2: [0, 4, 5], 3: [1, 6], 4: [1, 2, 6, 7], 5: [2, 7], 6: [3, 4, 8], 7: [4, 5, 8], 8: [6, 7]})


In [3]:
# sample tree object for bfs and dfs
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

def buildTree(data: tuple):
    if data == ():
        return None
    val, left_data, right_data = data
    res = TreeNode(val)
    res.left = buildTree(left_data)
    res.right = buildTree(right_data)
    return res
inputs = (4, (2, (1, (), ()), (3, (), ())), (6, (5, (), ()), (7, (), ())))
sample_tree = buildTree(inputs)

#sample tree:
#       4
#      / \
#     2   6
#    / \ / \
#   1  3 5  7


### Breath First Search ###
One of the simplest algorithms for searching a graph and the archetype for many important graph algorithms. The below BFS precedure assumes that the input graph G = (V, E) is represented using adjacency lists.
The algorithm also uses a first-in, first-out queue Q to manage the set of gray vertices.

$\;\;\;\;$BFS(G, S)

$\;\;\;\;\;\;\;\;$for each vertex u $\in$ G.V - {s}
    
$\;\;\;\;\;\;\;\;\;\;\;\;$ u.color = WHITE
        
$\;\;\;\;\;\;\;\;\;\;\;\;$ u.d = $\infty $

$\;\;\;\;\;\;\;\;\;\;\;\;$ u.$\pi$ = $NIL$

$\;\;\;\;\;\;\;\;$ s.color  = $GRAY$

$\;\;\;\;\;\;\;\;$ s.d = 0

$\;\;\;\;\;\;\;\;$ s.$\pi$ = $NIL$

$\;\;\;\;\;\;\;\;$ Q = $\emptyset\$

$\;\;\;\;\;\;\;\;$ ENQUEUE(Q, s)

$\;\;\;\;\;\;\;\;$ while Q $\neq$ $\emptyset\$

$\;\;\;\;\;\;\;\;\;\;\;\;$ u = DEQUEUE(Q)

$\;\;\;\;\;\;\;\;\;\;\;\;$ for each v $\in$ G.Adj[u]

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ if v.color == WHITE

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ v.color := GRAY

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ v.d = u.d + 1

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ v.$\pi$ = u

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ ENQUEUE(Q, v)

$\;\;\;\;\;\;\;\;\;\;\;\;$ u.color := $BLACK$



In [4]:
# BFS on tree
def bfs(root: TreeNode):
    if not root: return []
    res, queue = [], collections.deque([root])
    while queue:
        n = len(queue)
        level = []
        for _ in range(n):
            node = queue.popleft()
            level.append(node.val)
            queue += list(filter(None, [node.left, node.right]))
        res.append(level)
    return res
print(bfs(sample_tree))

[[4], [2, 6], [1, 3, 5, 7]]


In [5]:
# BFS on graph
def bfsGraph(graph: collections.defaultdict, start: int, n: int):
    # visit state, distance from starting point and predecessor
    state = {i:("unvisited", sys.maxsize, None) for i in range(n)}
    state[start], queue, res = ("visiting", 0, None), collections.deque([start]), []
    while len(queue) > 0:
        n, level = len(queue), []
        for _ in range(n):
            u = queue.popleft()
            status, distance, predecessor = state[u]
            for v in graph[u]:
                if state[v][0]!="unvisited":
                    continue
                state[v] = ("visiting", state[u][1] + 1, u)
                queue.append(v)
            state[u] = ("visited", distance, predecessor)
            level.append(u)
        res.append(level)
    return res
print(bfsGraph(graph, 0, 9))
print(bfsGraph(graph, 4, 9))

[[0], [1, 2], [3, 4, 5], [6, 7], [8]]
[[4], [1, 2, 6, 7], [0, 3, 5, 8]]


### Depth-first Search ###

Unlike breadth-first search, whose predecessor subgraph forms a tree, the predecessor subgraph produced by a depth first searchg may be composed of several trees, because the search may repeat from multiple sources. Therefore, we define the `predecessor subgraph` fof a depth-first search slightly differently from that of a breadth-first search: 
we let $G_{\pi}$ = ( $V$, $E_{\pi}$ ), where

$E_{\pi}$ = {(v.$pi$, v) : v$\in$V and v.$\pi$ $\neq$ $NIL$ }

The predecessor subgraph of a depth-first search forms a depth-first forest comprising several depth-first trees. The edges in $E_{\pi}$ are tree edges.

The following pseudocode is the basic depth-first-search algorithm. The input graph G may be undirected or directed. The variable time is a global variable that we use for timestamping.

$\;\;\;\;$ DFS(G)

$\;\;\;\;\;\;\;\;$ for each vertex u $\in$ G.V

$\;\;\;\;\;\;\;\;\;\;\;\;$ u.color := $WHITE$

$\;\;\;\;\;\;\;\;\;\;\;\;$ u.$\pi$ := $NIL$

$\;\;\;\;\;\;\;\;$ time = 0

$\;\;\;\;\;\;\;\;$ for each vertex u $\in$ G.V

$\;\;\;\;\;\;\;\;\;\;\;\;$ if u.color == WHITE

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ DFS-VISIT(G, u)

$\;\;\;\;$ DFS-VISIT(G, u)

$\;\;\;\;\;\;\;\;$ time = time + 1 //white vertex u has just been discovered

$\;\;\;\;\;\;\;\;$ u.d := time 

$\;\;\;\;\;\;\;\;$ u.color := $GRAY$

$\;\;\;\;\;\;\;\;$ for each v $\in$ G.Adj[u]

$\;\;\;\;\;\;\;\;\;\;\;\;$ if v.color == $WHITE$

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ v.$\pi$ := u

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ DFS-VISIT(G, v)

$\;\;\;\;\;\;\;\;$ u.color := $BLACK$

$\;\;\;\;\;\;\;\;$ time := time + 1

$\;\;\;\;\;\;\;\;$ u.f := time


In [79]:
# dfs on tree
def treeDFS():
    # preorder, inorder, postorder
    order = ["", "", ""]
    def dfs(root: TreeNode, order: List[str]):
        if not root:
            return None
        order[0] += str(root.val)
        dfs(root.left, order)
        order[1] += str(root.val)
        dfs(root.right, order)
        order[2] += str(root.val)
    dfs(sample_tree, order)
    print(order)
    
def treeDFSStack(root: TreeNode):
    stack, current = [], root
    preorder, inorder, postorder = "preorder: ", "inorder: ", "postorder: "
    while stack or current:
        while current:
            inorder += str(current.val)
            stack.append(current)
            current = current.left
        current = stack.pop()
        preorder += str(current.val)
        current = current.right
    stack, current, temp = [], root, collections.deque([])
    while stack or current:
        while current:
            temp.appendleft(str(current.val))
            stack.append(current)
            current = current.right
        current = stack.pop()
        current = current.left
    postorder += "".join(temp)
    print(inorder, preorder, postorder)
treeDFS()
treeDFSStack(sample_tree)

['4213657', '1234567', '1325764']
inorder: 4213657 preorder: 1234567 postorder: 1325764


In [89]:
def dfsGraph(graph: collections.defaultdict, n: int):
    # visit state, discovered_time, predecessor
    state = {i:["unvisited", sys.maxsize, None] for i in range(n)}
    time = [0]
    def dfs(root):
        time[0] = time[0] + 1
        state[root][0], state[root][1] = "visiting", time[0]
        for v in graph[root]:
            if state[v][0] == "unvisited":
                state[v][2] = root
                dfs(v)
        state[root][0] = "visited"
    for node in range(n):
        if state[node][0]=="unvisited":
            dfs(node)
    print(state)
dfsGraph(graph, 9)

{0: ['visited', 1, None], 1: ['visited', 2, 0], 2: ['visited', 6, 4], 3: ['visited', 3, 1], 4: ['visited', 5, 6], 5: ['visited', 7, 2], 6: ['visited', 4, 3], 7: ['visited', 8, 5], 8: ['visited', 9, 7]}


### Topological sort ###
This section shows how we can use depth-first search to perform a topological sort of a directed acyclic graph (DAG). A `topological sort` of a dag G = (V,E) is a linear ordering of all its vertices such that if G contains an edge (u, v), then u appears before v in the ordering.
The following simple algorithm topologically sorts a dag:

$\;\;\;\;$ TOPOLOGICAL-SORT(G)

$\;\;\;\;\;\;\;\;$ 1. call DFS(G) to compute finishing times v.f for each vertex v

$\;\;\;\;\;\;\;\;$ 2. as each vertex is finished , insert it onto the front of a linked list

$\;\;\;\;\;\;\;\;$ 3. return the linked list of vertices

Example: Course Schedule problem


### [207. Course Schedule ](https://leetcode.com/problems/course-schedule/)


There are a total of `numCourses` courses you have to take, labeled from `0` to `numCourses-1`.

Some courses may have prerequisites, for example to take course 0 you have to first take course 1, which is expressed as a pair: `[0,1]`

Given the total number of courses and a list of prerequisite pairs, is it possible for you to finish all courses?

#### Example 1:

Input: numCourses = 2, prerequisites = \[\[1,0\]\]

Output: true

Explanation: There are a total of 2 courses to take. 
             To take course 1 you should have finished course 0. So it is possible.

#### Example 2:

Input: numCourses = 2, prerequisites = \[\[1,0\],\[0,1\]\]

Output: false

Explanation: There are a total of 2 courses to take. 
             To take course 1 you should have finished course 0, and to take course 0 you should
             also have finished course 1. So it is impossible.

#### Constraints:

-   The input prerequisites is a graph represented by a list of edges, not adjacency matrices. Read more about [how a graph is represented](https://www.khanacademy.org/computing/computer-science/algorithms/graph-representation/a/representing-graphs).
-   You may assume that there are no duplicate edges in the input prerequisites.
-   `1 <= numCourses <= 10^5`

In [ ]:
class Solution:
    def findOrder(self, numCourses: int, prerequisites: List[List[int]]) -> List[int]:
        res, graph, state = collections.deque([]), collections.defaultdict(list), ["Not taking"]*numCourses
        for v, u in prerequisites:
            graph[u] += [v]
        def dfs(course):
            if state[course] == "taken":
                return True
            if state[course] == "taking":
                return False
            state[course] = "taking"
            for to_take in graph[course]:
                if not dfs(to_take):
                    return False
            state[course] = "taken"
            res.appendleft(course)
            return True
        for course in range(numCourses):
            if dfs(course) == False:
                return False
        return True

### [210. Course Schedule II](https://leetcode.com/problems/course-schedule-ii/)


There are a total of *n* courses you have to take, labeled from `0` to `n-1`.

Some courses may have prerequisites, for example to take course 0 you have to first take course 1, which is expressed as a pair: `[0,1]`

Given the total number of courses and a list of prerequisite pairs, return the ordering of courses you should take to finish all courses.

There may be multiple correct orders, you just need to return one of them. If it is impossible to finish all courses, return an empty array.

#### Example 1:

Input: 2, \[\[1,0\]\] 

Output: `[0,1]`

Explanation: There are a total of 2 courses to take. To take course 1 you should have finished   
             course 0. So the correct course order is `[0,1] .`

#### Example 2:

Input: 4, \[\[1,0\],\[2,0\],\[3,1\],\[3,2\]\]

Output: `[0,1,2,3] or [0,2,1,3]`

Explanation: There are a total of 4 courses to take. To take course 3 you should have finished both     
             courses 1 and 2. Both courses 1 and 2 should be taken after you finished course 0. 
             So one correct course order is `[0,1,2,3]`. Another correct ordering is `[0,2,1,3] .`

#### Note:

1.  The input prerequisites is a graph represented by a list of edges, not adjacency matrices. Read more about [how a graph is represented](https://www.khanacademy.org/computing/computer-science/algorithms/graph-representation/a/representing-graphs).

2.  You may assume that there are no duplicate edges in the input prerequisites.

In [ ]:
class Solution:
    def findOrder(self, numCourses: int, prerequisites: List[List[int]]) -> List[int]:
        res, graph, state = collections.deque([]), collections.defaultdict(list), ["Not taking"]*numCourses
        for v, u in prerequisites:
            graph[u] += [v]
        def dfs(course):
            if state[course] == "taken":
                return True
            if state[course] == "taking":
                return False
            state[course] = "taking"
            for to_take in graph[course]:
                if not dfs(to_take):
                    return False
            state[course] = "taken"
            res.appendleft(course)
            return True
        for course in range(numCourses):
            if dfs(course) == False:
                return []
        return res

### Shortest Paths ###

At the beginning of this section, we claimed that breadth-first search finds the distance to each reachable vertex in a graph G = (V, E) from a given source vertex s $\in$ V. Define the `shortest-path distance` $\delta$(s, v) from s to v as the minimum number of edges in any path from vertex ss to vertex v; if there is no path from s to v, then $\delta$(s, v)=$\infty$. We call a path of length $\delta$(s, v) from s to v a shortest path from s to v.

The following procedure prints out the vertices on a shortest path from s to v, assuming that BFS has already computed a breath-first tree

$\;\;\;\;$ PRINT-PATH(G, s, v)

$\;\;\;\;\;\;\;\;$ if v==s

$\;\;\;\;\;\;\;\;\;\;\;\;$ print s

$\;\;\;\;\;\;\;\;$ elseif v.$\pi$==$NIL$

$\;\;\;\;\;\;\;\;\;\;\;\;$ print "no path from" s "to" v "exists"

$\;\;\;\;\;\;\;\;$ else PRINT-PATH(G, s, v.$\pi$)

$\;\;\;\;\;\;\;\;\;\;\;\;$ print v


In [32]:
def printPath(graph, src, dest, state):
    _, _, predecessor = state[dest]
    if src == dest:
        return str(src)
    elif predecessor == None:
        print("No path from src to dest exists")
        return dest
    else:
        return str(printPath(graph, src, predecessor, state)) + " => " + str(dest)

### Dijkstra Algorithm ###


### [1514. Path with Maximum Probability](https://leetcode.com/problems/path-with-maximum-probability/)


You are given an undirected weighted graph of `n` nodes (0-indexed), represented by an edge list where `edges[i] = [a, b]` is an undirected edge connecting the nodes `a` and `b` with a probability of success of traversing that edge `succProb[i]`.

Given two nodes `start` and `end`, find the path with the maximum probability of success to go from `start` to `end` and return its success probability.

If there is no path from `start` to `end`, return 0. Your answer will be accepted if it differs from the correct answer by at most 1e-5.

#### Example 1:

![](https://assets.leetcode.com/uploads/2019/09/20/1558_ex1.png)

Input: n = 3, edges = \[\[0,1\],\[1,2\],\[0,2\]\], succProb = \[0.5,0.5,0.2\], start = 0, end = 2
Output: 0.25000
Explanation: There are two paths from start to end, one having a probability of success = 0.2 and the other has 0.5 \* 0.5 = 0.25.

#### Example 2:

![](https://assets.leetcode.com/uploads/2019/09/20/1558_ex2.png)

Input: n = 3, edges = \[\[0,1\],\[1,2\],\[0,2\]\], succProb = \[0.5,0.5,0.3\], start = 0, end = 2
Output: 0.30000

#### Example 3:

![](https://assets.leetcode.com/uploads/2019/09/20/1558_ex3.png)

Input: n = 3, edges = \[\[0,1\]\], succProb = \[0.5\], start = 0, end = 2
Output: 0.00000
Explanation: There is no path between 0 and 2.

#### Constraints:

-   `2 <= n <= 10^4`
-   `0 <= start, end < n`
-   `start != end`
-   `0 <= a, b < n`
-   `a != b`
-   `0 <= succProb.length == edges.length <= 2*10^4`
-   `0 <= succProb[i] <= 1`
-   There is at most one edge between every two nodes.

In [90]:
class Solution:
    def maxProbability(self, n: int, edges: List[List[int]], succProb: List[float], start: int, end: int) -> float:
        def makeGraph():
            graph, nodes = collections.defaultdict(list),set()
            for i in range(len(edges)):
                x, y = edges[i][0], edges[i][1]
                graph[x] += [(y, succProb[i])]
                graph[y] += [(x, succProb[i])]
                nodes |= {x, y}
            return graph, nodes
        graph, nodes = makeGraph()
        '''
        # bfs with greedy approach
        if end not in nodes: return 0.0
        probs = [0.0 if i!=start else 1.0 for i in range(n)]
        level = collections.deque([start])
        while level:
            node = level.popleft()
            for v, p in graph[node]:
                if probs[node]*p > probs[v]:
                    level.append(v)
                    probs[v] = probs[node]*p
        return probs[end]
        '''
        # Dijkstra with priority queue
        level = [(-1, start)]
        visited = set()
        while level:
            p, node = heapq.heappop(level)
            if node == end: return -p
            visited.add(node)
            for neighbor, pr in graph[node]:
                if neighbor in visited:
                    continue
                heapq.heappush(level, (p*pr, neighbor))
        return 0.0
    

### [787. Cheapest Flights Within K Stops](https://leetcode.com/problems/cheapest-flights-within-k-stops/)

There are `n` cities connected by `m` flights. Each flight starts from city `u` and arrives at `v` with a price `w`.

Now given all the cities and flights, together with starting city `src` and the destination `dst`, your task is to find the cheapest price from `src` to `dst` with up to `k` stops. If there is no such route, output `-1`.

#### Example 1:
Input: 
n = 3, edges = \[\[0,1,100\],\[1,2,100\],\[0,2,500\]\]

src = 0, dst = 2, k = 1

Output: 200

Explanation: 
The graph looks like this:
![](https://s3-lc-upload.s3.amazonaws.com/uploads/2018/02/16/995.png)

The cheapest price from city `0` to city `2` with at most 1 stop costs 200, as marked red in the picture.

#### Example 2:
Input: 
n = 3, edges = \[\[0,1,100\],\[1,2,100\],\[0,2,500\]\]

src = 0, dst = 2, k = 0

Output: 500

Explanation: 
The graph looks like this:
![](https://s3-lc-upload.s3.amazonaws.com/uploads/2018/02/16/995.png)

The cheapest price from city `0` to city `2` with at most 0 stop costs 500, as marked blue in the picture.

#### Constraints:

-   The number of nodes `n` will be in range `[1, 100]`, with nodes labeled from `0` to `n`` - 1`.
-   The size of `flights` will be in range `[0, n * (n - 1) / 2]`.
-   The format of each flight will be `(src, ``dst``, price)`.
-   The price of each flight will be in the range `[1, 10000]`.
-   `k` is in the range of `[0, n - 1]`.
-   There will not be any duplicated flights or self cycles.

In [92]:
class Solution:
    def findCheapestPrice(self, n: int, flights: List[List[int]], src: int, dst: int, K: int) -> int:
        pq, graph = [(0, src, K)], collections.defaultdict(list)
        for src, dest, weight in flights:
            graph[src]+=[(dest, weight)]
        while pq:
            cost_to_src, src, stop_left = heapq.heappop(pq)
            if src == dst:
                return cost_to_src
            if stop_left < 0:
                continue
            for dest, weight in graph[src]:
                heapq.heappush(pq, (cost_to_src+weight, dest, stop_left-1))
        return -1

### Bellmanford Algorithm ###


### [743. Network Delay Time](https://leetcode.com/problems/network-delay-time/)

There are `N` network nodes, labelled `1` to `N`.

Given `times`, a list of travel times as directed edges `times[i] = (u, v, w)`, where `u` is the source node, `v` is the target node, and `w` is the time it takes for a signal to travel from source to target.

Now, we send a signal from a certain node `K`. How long will it take for all nodes to receive the signal? If it is impossible, return `-1`.

Example 1:

![](https://assets.leetcode.com/uploads/2019/05/23/931_example_1.png)

Input: times = \[\[2,1,1\],\[2,3,1\],\[3,4,1\]\], N = 4, K = 2
Output: 2

#### Note:

1.  `N` will be in the range `[1, 100]`.
2.  `K` will be in the range `[1, N]`.
3.  The length of `times` will be in the range `[1, 6000]`.
4.  All edges `times[i] = (u, v, w)` will have `1 <= u, v <= N` and `0 <= w <= 100`.

In [91]:
class Solution:
    def networkDelayTime(self, times: List[List[int]], N: int, K: int) -> int:
        dist = {i:sys.maxsize for i in range(N)}
        dist[K-1] = 0
        for node in range(N):
            for u, v, weight in times:
                if dist[u-1]!=sys.maxsize and dist[v-1] > dist[u-1] + weight:
                    dist[v-1] = dist[u-1] + weight
        res = max(dist.values())
        return -1 if res==sys.maxsize else res

### Strongly connected components  (Union find) ###

A classic application of depth-first search: decomposing a directed graph into its strongly connected components. This section shows how to do so using two depth-first searches. Many algorithms that work with directed graphs begin with such a decomposition.

A strongly connected component of a directed graph G = (V, E) is a maximal set of vertices C $\subseteq$ V such that for every pair of vertices u and v in C, we have both u $\rightarrow$ v and v $\rightarrow$ u; that is, vertices u and v are reachable from each other.

### Tarjan's algorithm ###
$\;\;\;\;$ STRONGLY-CONNECTED-COMPONENTS(G)

$\;\;\;\;\;\;\;\;$ 1. call DFS(G) to compute finishing times u.f for each vertex u

$\;\;\;\;\;\;\;\;$ 2. compute $G^{T}$

$\;\;\;\;\;\;\;\;$ 3. call DFS($G^{T}$), but in the main loop of DFS. consider the vertices in order of decreasing u.f (as computed in line 1)

$\;\;\;\;\;\;\;\;$ 4. output the vertices of each tree in the depth-first forest formed in line 3 as a separate strongly connected component.

### Example ###

[684. Redundant Connection](https://leetcode.com/problems/redundant-connection/)

[685. Redundant Connection II](https://leetcode.com/problems/redundant-connection-ii/)

[1192. Critical Connections in a Network](https://leetcode.com/problems/critical-connections-in-a-network/)


### [1192. Critical Connections in a Network](https://leetcode.com/problems/critical-connections-in-a-network/)

There are `n` servers numbered from `0` to `n-1` connected by undirected server-to-server `connections` forming a network where `connections[i] = [a, b]` represents a connection between servers `a` and `b`. Any server can reach any other server directly or indirectly through the network.

A *critical connection* is a connection that, if removed, will make some server unable to reach some other server.

Return all critical connections in the network in any order.

#### Example 1:

![](https://assets.leetcode.com/uploads/2019/09/03/1537_ex1_2.png)

Input: n = 4, connections = \[\[0,1\],\[1,2\],\[2,0\],\[1,3\]\]
Output: \[\[1,3\]\]
Explanation: \[\[3,1\]\] is also accepted.

#### Constraints:

-   `1 <= n <= 10^5`
-   `n-1 <= connections.length <= 10^5`
-   `connections[i][0] != connections[i][1]`
-   There are no repeated connections.

In [ ]:
class Solution(object):
    def criticalConnections(self, n, connections):
        """
        :type n: int
        :type connections: List[List[int]]
        :rtype: List[List[int]]
        """
        connect = collections.defaultdict(list)
        for u,v in connections:
            connect[u] += [v]
            connect[v] += [u]
        self.dep = 0
        self.res = []
        depth, lowest, parent, visited = [float("inf")]*n, [float("inf")]*n, [float("inf")]*n, [False]*n
        def dfs(u):
            visited[u] = True
            depth[u], lowest[u] = self.dep, self.dep
            self.dep += 1
            for v in connect[u]:   
                if not visited[v]:
                    parent[v] = u
                    dfs(v)
                if parent[u]!=v:
                    lowest[u] = min(lowest[u], lowest[v])   
                    # upgrade the lowest so strongly connected graph has the same lowest
                if lowest[v]>depth[u]:
                    self.res.append([u,v])
        dfs(0)
        return self.res

### [684. Redundant Connection](https://leetcode.com/problems/redundant-connection/)


In this problem, a tree is an undirected graph that is connected and has no cycles.

The given input is a graph that started as a tree with N nodes (with distinct values 1, 2, ..., N), with one additional edge added. The added edge has two different vertices chosen from 1 to N, and was not an edge that already existed.

The resulting graph is given as a 2D-array of `edges`. Each element of `edges` is a pair `[u, v]` with `u < v`, that represents an undirected edge connecting nodes `u` and `v`.

Return an edge that can be removed so that the resulting graph is a tree of N nodes. If there are multiple answers, return the answer that occurs last in the given 2D-array. The answer edge `[u, v]` should be in the same format, with `u < v`.

#### Example 1:  

Input: \[\[1,2\], \[1,3\], \[2,3\]\]

Output: \[2,3\]

Explanation: The given undirected graph will be like this:

    1
    /  \
    2  -  3

#### Example 2:  

Input: \[\[1,2\], \[2,3\], \[3,4\], \[1,4\], \[1,5\]\]

Output: \[1,4\]

Explanation: The given undirected graph will be like this:

5 -  1 - 2

    |   |
    
    4 - 3

#### Note:  

-   The size of the input 2D-array will be between 3 and 1000.

-   Every integer represented in the 2D-array will be between 1 and N, where N is the size of the input array.

  

#### Update (2017-09-26):  

We have overhauled the problem description + test cases and specified clearly the graph is an *undirected* graph. For the *directed* graph follow up please see [Redundant Connection II](https://leetcode.com/problems/redundant-connection-ii/description/)). We apologize for any inconvenience caused.

In [ ]:
class Solution:
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        self.parent = [i for i in range(2001)]
        def find(node):
            if node!=self.parent[node]:
                self.parent[node] = find(self.parent[node])
            return self.parent[node]
        for u, v in edges:
            u_parent, v_parent = find(u), find(v)
            if u_parent == v_parent: return [u, v]
            else: self.parent[v_parent] = u_parent
        return [0, 0]

### [685. Redundant Connection II](https://leetcode.com/problems/redundant-connection-ii/)

In this problem, a rooted tree is a directed graph such that, there is exactly one node (the root) for which all other nodes are descendants of this node, plus every node has exactly one parent, except for the root node which has no parents.

The given input is a directed graph that started as a rooted tree with N nodes (with distinct values 1, 2, ..., N), with one additional directed edge added. The added edge has two different vertices chosen from 1 to N, and was not an edge that already existed.

The resulting graph is given as a 2D-array of `edges`. Each element of `edges` is a pair `[u, v]` that represents a directed edge connecting nodes `u` and `v`, where `u` is a parent of child `v`.

Return an edge that can be removed so that the resulting graph is a rooted tree of N nodes. If there are multiple answers, return the answer that occurs last in the given 2D-array.

#### Example 1:  

Input: \[\[1,2\], \[1,3\], \[2,3\]\]

Output: \[2,3\]



#### Example 2:  

Input: \[\[1,2\], \[2,3\], \[3,4\], \[4,1\], \[1,5\]\]

Output: \[4,1\]

Explanation: The given directed graph will be like this:

5 <- 1   -> 2

     ^    |
     |    v
     
     4 <- 3

#### Note:  

-   The size of the input 2D-array will be between 3 and 1000.
-   Every integer represented in the 2D-array will be between 1 and N, where N is the size of the input array.

In [ ]:
class Solution:
    def findRedundantDirectedConnection(self, edges: List[List[int]]) -> List[int]:
        N, parent, candidates = len(edges), {}, []
        for u, v in edges:
            if v in parent:
                candidates.append((parent[v], v))
                candidates.append((u, v))
            else:
                parent[v] = u

        def orbit(node):
            seen = set()
            while node in parent and node not in seen:
                seen.add(node)
                node = parent[node]
            return node, seen

        root = orbit(1)[0]

        if not candidates:
            cycle = orbit(root)[1]
            for u, v in edges:
                if u in cycle and v in cycle:
                    ans = u, v
            return ans

        children = collections.defaultdict(list)
        for v in parent:
            children[parent[v]].append(v)

        seen = [True] + [False] * N
        stack = [root]
        while stack:
            node = stack.pop()
            if not seen[node]:
                seen[node] = True
                stack.extend(children[node])

        return candidates[all(seen)]

### Minimum Spanning Tree ###

### [1489. Find Critical and Pseudo-Critical Edges in Minimum Spanning Tree](https://leetcode.com/problems/find-critical-and-pseudo-critical-edges-in-minimum-spanning-tree/)

Given a weighted undirected connected graph with `n` vertices numbered from `0` to `n-1`, and an array `edges` where `edges[i] = [fromi, toi, weighti]` represents a bidirectional and weighted edge between nodes `fromi` and `toi`. A minimum spanning tree (MST) is a subset of the edges of the graph that connects all vertices without cycles and with the minimum possible total edge weight.

Find *all the critical and pseudo-critical edges in the minimum spanning tree (MST) of the given graph*. An MST edge whose deletion from the graph would cause the MST weight to increase is called a *critical edge*. A *pseudo-critical edge*, on the other hand, is that which can appear in some MSTs but not all.

Note that you can return the indices of the edges in any order.

#### Example 1:

![](https://assets.leetcode.com/uploads/2020/06/04/ex1.png)

Input: n = 5, edges = \[\[0,1,1\],\[1,2,1\],\[2,3,2\],\[0,3,2\],\[0,4,3\],\[3,4,3\],\[1,4,6\]\]

Output: \[\[0,1\],\[2,3,4,5\]\]

Explanation: The figure above describes the graph.
The following figure shows all the possible MSTs:
![](https://assets.leetcode.com/uploads/2020/06/04/msts.png)
Notice that the two edges 0 and 1 appear in all MSTs, therefore they are critical edges, so we return them in the first list of the output.
The edges 2, 3, 4, and 5 are only part of some MSTs, therefore they are considered pseudo-critical edges. We add them to the second list of the output.

#### Example 2:

![](https://assets.leetcode.com/uploads/2020/06/04/ex2.png)

Input: n = 4, edges = \[\[0,1,1\],\[1,2,1\],\[2,3,1\],\[0,3,1\]\]

Output: \[\[\],\[0,1,2,3\]\]

Explanation: We can observe that since all 4 edges have equal weight, choosing any 3 edges from the given 4 will yield an MST. Therefore all 4 edges are pseudo-critical.

#### Constraints:

-   `2 <= n <= 100`
-   `1 <= edges.length <= min(200, n * (n - 1) / 2)`
-   `edges[i].length == 3`
-   `0 <= fromi < toi < n`
-   `1 <= weighti <= 1000`
-   All pairs `(fromi, toi)` are distinct.

In [93]:
class UnionFind:
    def __init__(self, n):
        self.parent = [i for i in range(n)]

    def find(self, a):
        if self.parent[a] == a:
            return a
        self.parent[a] = self.find(self.parent[a])
        return self.parent[a]

    def union(self, a, b):
        parent_a, parent_b = self.find(a), self.find(b)
        if parent_a == parent_b:
            return False
        self.parent[max(parent_a, parent_b)] = min(parent_a, parent_b)
        return True
    
class Solution:
    def findCriticalAndPseudoCriticalEdges(self, n: int, edges: List[List[int]]) -> List[List[int]]:
        self.graph = self.makeGraph(edges)
        ref = self.Kruskal(n)
        critical, pseudo = [], []
        for i in range(len(edges)):
            u, v, w = edges[i]
            if self.Kruskal(n, skip = (u, v)) > ref:
                critical.append(i)
            elif self.Kruskal(n, pick=(u, v, w))==ref:
                pseudo.append(i)
        return [critical, pseudo]
                
    def makeGraph(self, edges: List[List[int]]) -> collections.defaultdict:
        graph = collections.defaultdict(list)
        for u, v, weight in edges:
            graph[u] += [(v, weight)]
            graph[v] += [(u, weight)]
        return graph
        
    def Kruskal(self, n: int, pick = None, skip = None)->int:
        res, pq = 0, []
        uf = UnionFind(n)
        if pick:
            u, v, w = pick
            res += w
            n -= 1
            uf.union(u, v)
        for u in self.graph:
            for v, w in self.graph.get(u, []):
                if skip and u in skip and v in skip: 
                    continue
                heapq.heappush(pq, (w, u, v))
        while pq:
            w, u, v = heapq.heappop(pq)
            if uf.find(u)!=uf.find(v):
                uf.union(u, v)
                n -= 1
                res += w
        return res if n <= 1 else inf
    
    # return the weight of MST using Prim's algorithm
    def mstPrim(self, n: int, pick = None, skip = None)->int:
        # Mark node and put its edges to priority queue
        def visit(u):
            visited[u] = True
            for v, w in self.graph.get(u, []):
                if skip and u in skip and v in skip: 
                    continue
                if not visited[v]: heappush(pq, (w, u, v))
        res, visited, pq = 0, [False]*n, []
        if pick:
            u, v, w = pick
            res += w
            visited[u], visited[v] = True, True
            visit(u)
            visit(v)
        else:
            visit(0)
        while pq:
            w, u, v = heappop(pq)
            if visited[u] and visited[v]: continue
            res += w
            if not visited[u]: visit(u)
            if not visited[v]: visit(v)
        return res if all(visited) else inf